In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatOpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7,
)

prompt1 = ChatPromptTemplate.from_messages([
    ("system", "당신은 신뢰할 수 있는 한국어 여행 가이드입니다. "
               "질문이 모호하더라도 사용자가 지정한 여행지(도시/국가)에 대해 대표 관광 명소 1가지만 한국어로 추천하세요. "
               "가능하면 고유명사만 출력하세요."),
    ("user", "{destination}의 대표적인 관광 명소 한 가지를 추천해 주세요. "
             "반드시 명소명만 출력해 주세요.")
])
chain1 = prompt1 | llm | StrOutputParser()

prompt2 = ChatPromptTemplate.from_messages([
    ("system", "너는 간결하고 정확한 한국어 여행 도슨트다. "
               "사실과 일반적 방문 팁을 구조화해 설명하라."),
    ("user",
     "다음 명소에 대한 상세 정보를 한국어로 정리해 주세요.\n"
     "명소: {attraction}\n\n"
     "형식은 아래를 따르세요.\n"
     "명소: {attraction}\n"
     "역사:\n- ...\n- ...\n"
     "특징:\n- ...\n- ...\n"
     "방문 팁:\n- ...\n- ...")
])
chain2 = prompt2 | llm | StrOutputParser()

combined_chain = (
    {"destination": RunnablePassthrough()}
    | RunnablePassthrough.assign(attraction=chain1)
    | RunnablePassthrough.assign(details=chain2)
)

destination = "로마"
result = combined_chain.invoke({"destination": destination})

print(f'입력: "{destination}"')
print(f'1단계 결과: "{result["attraction"]}"')
print("2단계 결과:")
print(f"명소: {result['attraction']}\n{result['details']}")

입력: "로마"
1단계 결과: "콜로세움"
2단계 결과:
명소: 콜로세움
명소: 콜로세움  
역사:  
- AD 72년 베스파시아누스 황제가 착공, AD 80년 티투스 황제가 완공(8년 공사).  
- 고대 로마 최대 원형경기장으로 약 5만 명 수용, 검투사·야수·모의해전(나우마키아) 등 공연.  
- 6세기까지 사용 후 지진·석재 약탈로 붕괴, 18세기 토머스 콕 등 대(Grand Tour) 시대부터 관광지화.  

특징:  
- 타원형(외경 189×156 m, 높이 48 m) 4층 구조: 1·2층 아치·기둥 장식, 3층 이온·코린트·코린트 혼용, 4층 청동 방패창.  
- 지하(히포게움)는 무대 장치·야수 우리·승강기·슈어리(수중전용) 수조 유적 보존.  
- 1997년 유네스코 세계문화유산 등재, 이탈리아 국보급 건축물.  

방문 팁:  
- 온라인 예약 필수(현장 구매보다 2€ 수수료 있어도 대기 시간 절약), 이른 아침·저녁 1시간 전 입장이 빠름.  
- “Full Experience” 티켓(히포게움+경기장 바닥) 구매 시 해설 오디오 가이드 포함, 야간 투어(4~10월) 분위기 좋음.  
- 경찰 단속 강화: 밖에 앉아서 피크닉·식사 금지, 가방은 X-ray 검색, 칼·가위류 반입 불가.
